In [ ]:
import datetime as dt
import pandas as pd
import redis
import numpy as np

from fastcounting import helper, views, store, files, system

In [ ]:
r = redis.Redis(**helper.Helper().rediscred, decode_responses=True)

In [ ]:
# unique list of all accounts we have at least one atomic transactions linked to
# its a lot faster to make the list unique first

In [ ]:
lua_accounts = """
local result = {}
local hash = {}
for i, list in ipairs(redis.call(
    'ZRANGEBYSCORE', KEYS[1], ARGV[1], ARGV[2], ARGV[3])) do
if i%2==0 and not hash[list] then
result[#result+1] = list
hash[list] = true
end
end
return result
"""

In [ ]:
response = r.eval(lua_accounts, 1, 'account:atomic', 0, 999999, 'WITHSCORES')

In [ ]:
# lets build on lua_accounts

In [ ]:
# i am a lua beginner here is what i did
# first i looped over all values with scores (account number) in 'account:atomic'
# i made a unique array from all the scores or from the range of scores you specified
# second i loop over the same store again but now only one account a time
# i use the atomics from this second loop to lookup their amount in the according hash
# i sum everything together and create a new item in the list result for every account
# thirdly i concat the two lists from 1 and 2
# so you use the final list somehow like:
# sums = dict(response[middle:], zip(response[:middle]))

lua_sum = """
local accounts = {}
local hash = {}
for i, list in ipairs(redis.call(
    'ZRANGEBYSCORE', KEYS[1], ARGV[1], ARGV[2], ARGV[3])) do
if i%2==0 and not hash[list] then
accounts[#accounts+1] = list
hash[list] = true
end
end

local result = {}
for idx, account in ipairs(accounts) do
result[idx] = 0
for i, atomic in pairs(redis.call(
    'ZRANGEBYSCORE', KEYS[1], account, account)) do
result[idx] = result[idx] + redis.call('HGET', 'atomicID:' .. atomic, 'amount')
end
end

for i=1,#accounts do
    result[#result+1] = accounts[i]
end
return result
"""

In [ ]:
response = r.eval(lua_sum, 1, 'account:atomic', 0, 9999999, 'WITHSCORES')

In [ ]:
r.eval("""
local accounti = {}
accounti = redis.call('hgetall', '1')
return accounti['amount']""", 0, 2, 'art')

In [ ]:
atomic_rows = """
local row = {}
for i, atomic in pairs(redis.call(
    'ZRANGEBYSCORE', KEYS[1], ARGV[1], ARGV[2])) do
row[i] = redis.call('HGET', 'atomicID:' .. atomic, 'accountID')
end
return row
"""
response = r.eval(atomic_rows, 1, 'atomic:date', 0, 99999999999999999)

# Those new stream views especially the stable account one.

In [ ]:
lua_stable_view = """
local row = {}
local generali = {}
local accounti = {}
local debug = {}
local account = 0.0
local general = 0.0
for i, atomic in pairs(redis.call(
    'ZRANGEBYSCORE', KEYS[1], ARGV[1], ARGV[2])) do
row = redis.call('HGETALL', 'atomicID:' .. atomic)
general = row[2]
account = row[6]
generali = redis.call('HGETALL', 'generalID:' .. general)
accounti = redis.call('HGETALL', 'accountsystem:' .. account)
if next(accounti)==nil then
    accounti = redis.call('HGETALL', 'accountsystem:special_account') end
redis.call('XADD','account:' .. account, '*', row[1], row[2], row[5], row[6], generali[1], generali[2])
end
return true
"""

In [ ]:
# arguments are dates, we don't need accounts because we create a stream for every account
response = r.eval(lua_stable_view, 1, 'atomic:date', 0, 99999999999999999)

In [ ]:
response

In [ ]:
r.hgetall('accountsystem:5906')

In [ ]:
r.hgetall('atomicID:')

In [ ]:
response = r.xrevrange('account:1830')

In [ ]:
response

In [ ]:
r.re